# Covariance and Correlation

Generates a covariance or Pearson correlation matrix for pairs of numeric columns in a table. Grouping added in 1.15.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.4.0 on GCP (demo machine)
%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum Database 4.3.10.0
#%sql postgresql://gpdbchina@10.194.10.68:61000/madlib

u'Connected: gpadmin@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.15-dev, git revision: rc/1.14-rc1-6-g3b80a32, cmake configuration time: Wed May 16 19:29:52 UTC 2018, build type: release, build system: Linux-2.6.32-696.20.1.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7"


# 1. Load data

In [4]:
%%sql
DROP TABLE IF EXISTS example_data CASCADE;

CREATE TABLE example_data( 
    id SERIAL, 
    outlook TEXT,
    temperature FLOAT8, 
    humidity FLOAT8,
    windy TEXT, 
    class TEXT,
    day TEXT
);

INSERT INTO example_data VALUES
(1, 'sunny', 85, 85, 'false', 'Dont Play', 'Mon'),
(2, 'sunny', 80, 90, 'true', 'Dont Play', 'Mon'),
(3, 'overcast', 83, 78, 'false', 'Play', 'Mon'),
(4, 'rain', 70, 96, 'false', 'Play', 'Mon'),
(5, 'rain', 68, 80, 'false', 'Play', 'Mon'),
(6, 'rain', 65, 70, 'true', 'Dont Play', 'Mon'),
(7, 'overcast', 64, 65, 'true', 'Play', 'Mon'),
(8, 'sunny', 72, 95, 'false', 'Dont Play', 'Mon'),
(9, 'sunny', 69, 70, 'false', 'Play', 'Mon'),
(10, 'rain', 75, 80, 'false', 'Play', 'Mon'),
(11, 'sunny', 75, 70, 'true', 'Play', 'Mon'),
(12, 'overcast', 72, 90, 'true', 'Play', 'Mon'),
(13, 'overcast', 81, 75, 'false', 'Play', 'Mon'),
(14, 'rain', 71, 80, 'true', 'Dont Play', 'Mon'),
(15, NULL, 100, 100, 'true', NULL, 'Mon'),
(16, NULL, 110, 100, 'true', NULL, 'Mon'),
(101, 'sunny', 85, 85, 'false', 'Dont Play', 'Tues'),
(102, 'sunny', 80, 90, 'true', 'Dont Play', 'Tues'),
(103, 'overcast', 83, 78, 'false', 'Play', 'Tues'),
(104, 'rain', 70, 96, 'false', 'Play', 'Tues'),
(105, 'rain', 68, 80, 'false', 'Play', 'Tues'),
(106, 'rain', 65, 70, 'true', 'Dont Play', 'Tues'),
(107, 'overcast', 64, 65, 'true', 'Play', 'Tues'),
(108, 'sunny', 72, 95, 'false', 'Dont Play', 'Tues'),
(109, 'sunny', 69, 70, 'false', 'Play', 'Tues'),
(110, 'rain', 75, 80, 'false', 'Play', 'Tues'),
(111, 'sunny', 75, 70, 'true', 'Play', 'Tues'),
(112, 'overcast', 72, 90, 'true', 'Play', 'Tues'),
(113, 'overcast', 81, 75, 'false', 'Play', 'Tues'),
(114, 'rain', 71, 80, 'true', 'Dont Play', 'Tues'),
(115, NULL, 100, 100, 'true', NULL, 'Tues'),
(116, NULL, 110, 100, 'true', NULL, 'Tues'),
(201, 'sunny', 85, 85, 'false', 'Dont Play', 'Wed'),
(202, 'sunny', 80, 90, 'true', 'Dont Play', 'Wed'),
(203, 'overcast', 83, 78, 'false', 'Play', 'Wed'),
(204, 'rain', 70, 96, 'false', 'Play', 'Wed'),
(205, 'rain', 68, 80, 'false', 'Play', 'Wed'),
(206, 'rain', 65, 70, 'true', 'Dont Play', 'Wed'),
(207, 'overcast', 64, 65, 'true', 'Play', 'Wed'),
(208, 'sunny', 7, 95, 'false', 'Dont Play', 'Wed'),
(209, 'sunny', 6, 70, 'false', 'Play', 'Wed'),
(210, 'rain', 7, 80, 'false', 'Play', 'Wed'),
(211, 'sunny', 75, 70, 'true', 'Play', 'Wed'),
(212, 'overcast', 72, 90, 'true', 'Play', 'Wed'),
(213, 'overcast', 81, 75, 'false', 'Play', 'Wed'),
(214, 'rain', 71, 80, 'true', 'Dont Play', 'Wed'),
(215, NULL, 10, 100, 'true', NULL, 'Wed'),
(216, NULL, 10, 100, 'true', NULL, 'Wed'),
(217, 'sunny', 85, 85, 'false', 'Dont Play', 'Wed'),
(218, 'sunny', 80, 9, 'true', 'Dont Play', 'Wed'),
(219, 'overcast', 83, 78, 'false', 'Play', 'Wed'),
(220, 'rain', 70, 9, 'false', 'Play', 'Wed'),
(221, 'rain', 68, 80, 'false', 'Play', 'Wed');

SELECT * FROM example_data ORDER BY id;

Done.
Done.
53 rows affected.
53 rows affected.


id,outlook,temperature,humidity,windy,class,day
1,sunny,85.0,85.0,false,Dont Play,Mon
2,sunny,80.0,90.0,true,Dont Play,Mon
3,overcast,83.0,78.0,false,Play,Mon
4,rain,70.0,96.0,false,Play,Mon
5,rain,68.0,80.0,false,Play,Mon
6,rain,65.0,70.0,true,Dont Play,Mon
7,overcast,64.0,65.0,true,Play,Mon
8,sunny,72.0,95.0,false,Dont Play,Mon
9,sunny,69.0,70.0,false,Play,Mon
10,rain,75.0,80.0,false,Play,Mon


# 2. Correlation
Get correlation between temperature and humidity:

In [5]:
%%sql
DROP TABLE IF EXISTS example_data_output, example_data_output_summary;
SELECT madlib.correlation( 'example_data',
                           'example_data_output',
                           'temperature, humidity'
                         );

Done.
1 rows affected.


correlation
"Summary for 'Correlation' functionOutput table = example_data_outputProducing correlation for columns: temperature,humidityTotal run time = ('example_data_output', 2, 0.26578783988952637)"


View the correlation matrix:

In [6]:
%%sql
SELECT * FROM example_data_output ORDER BY column_position; 

2 rows affected.


column_position,variable,temperature,humidity
1,temperature,1.0,None
2,humidity,0.00607993890409,1.0


View the summary table:

In [7]:
%%sql
SELECT * FROM example_data_output_summary;

1 rows affected.


method,source,output_table,column_names,mean_vector,total_rows_processed
Correlation,example_data,example_data_output,"[u'temperature', u'humidity']","[70.188679245283, 79.8679245283019]",53


# 3.  Correlation with grouping
Run the correlation by day:

In [8]:
%%sql
DROP TABLE IF EXISTS example_data_output, example_data_output_summary;
SELECT madlib.correlation( 'example_data',
                           'example_data_output',
                           'temperature, humidity',
                           FALSE,
                           'day'
                         );

Done.
1 rows affected.


correlation
"Summary for 'Correlation' functionOutput table = example_data_outputGrouping columns: dayProducing correlation for columns: temperature,humidityTotal run time = ('example_data_output', 2, 0.22429895401000977)"


View the correlation matrix by group:

In [9]:
%%sql
SELECT * FROM example_data_output ORDER BY day, column_position;

6 rows affected.


column_position,variable,day,temperature,humidity
1,temperature,Mon,1.0,None
2,humidity,Mon,0.616876934549,1.0
1,temperature,Tues,1.0,None
2,humidity,Tues,0.616876934549,1.0
1,temperature,Wed,1.0,None
2,humidity,Wed,-0.289696693685,1.0


View the summary table:

In [11]:
%%sql
SELECT * FROM example_data_output_summary ORDER BY day;

3 rows affected.


method,source,output_table,column_names,day,mean_vector,total_rows_processed
Correlation,example_data,example_data_output,"[u'temperature', u'humidity']",Mon,"[77.5, 82.75]",16
Correlation,example_data,example_data_output,"[u'temperature', u'humidity']",Tues,"[77.5, 82.75]",16
Correlation,example_data,example_data_output,"[u'temperature', u'humidity']",Wed,"[59.0476190476191, 75.4761904761905]",21


# 4. Covariance
Get covariance between temperature and humidity:

In [13]:
%%sql
DROP TABLE IF EXISTS example_data_output, example_data_output_summary;
SELECT madlib.covariance( 'example_data',
                          'example_data_output',
                          'temperature, humidity'
                         );

Done.
1 rows affected.


covariance
"Summary for 'Covariance' functionOutput table = example_data_outputProducing covariance for columns: temperature,humidityTotal run time = ('example_data_output', 2, 0.1925361156463623)"


View covariance matrix:

In [14]:
%%sql
SELECT * FROM example_data_output ORDER BY column_position; 

2 rows affected.


column_position,variable,temperature,humidity
1,temperature,507.926664293,None
2,humidity,2.40227839089,307.35991456


View the summary table:

In [15]:
%%sql
SELECT * FROM example_data_output_summary;

1 rows affected.


method,source,output_table,column_names,mean_vector,total_rows_processed
Covariance,example_data,example_data_output,"[u'temperature', u'humidity']","[70.188679245283, 79.8679245283019]",53
